# Multi-Source Geospatial Data Integration

This notebook demonstrates integrating multiple geospatial data sources based on location:
- **NAIP**: USDA aerial imagery (1m resolution RGB)
- **SRTM**: Elevation data via Open-Elevation API

Goal: Show how different resolution datasets can be combined for spatial analysis.

In [ ]:
!pip install rasterio requests pillow matplotlib numpy

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import rasterio
import requests
from rasterio.transform import from_bounds
from rasterio.warp import reproject, Resampling

data_dir = Path('fire_data')
data_dir.mkdir(exist_ok=True)


## Part 1: Define Study Area


In [ ]:
# Study area options:
# Option 1: Los Angeles / Ventura County
# bbox = [-119.151962, 33.697190, -117.779357, 34.382245]

# Option 2: San Diego County (inland areas)
# bbox = [-117.429942, 32.619954, -116.743640, 32.967435]

# Option 3: Santa Monica Mountains
bbox = [-118.866603, 33.999553, -118.375308, 34.250224]

print(f"Study Area Bounding Box: {bbox}")
print(f"Longitude: {bbox[0]:.2f} to {bbox[2]:.2f}")
print(f"Latitude: {bbox[1]:.2f} to {bbox[3]:.2f}")

## Part 2: Download NAIP Aerial Imagery

NAIP provides high-resolution (1m) aerial imagery from USDA.

In [ ]:
def get_naip_imagery(bbox, output_path):
    """
    Fetch NAIP imagery - simple version
    Always gets 1024x1024 pixels, no calculations
    """
    base_url = "https://imagery.nationalmap.gov/arcgis/rest/services/USGSNAIPImagery/ImageServer/exportImage"

    params = {
        'bbox': f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
        'bboxSR': '4326',
        'size': '1024, 1024',
        'imageSR': '4326',
        'format': 'tiff',  # Get TIFF to preserve all bands
        'pixelType': 'U16',
        'f': 'image'
    }

    print(f"Requesting NAIP imagery: 1024x1024 pixels")
    response = requests.get(base_url, params=params, timeout=120)

    if response.status_code == 200:
        # Save directly - no conversion needed
        with open(output_path, 'wb') as f:
            f.write(response.content)

        print(f"✓ NAIP imagery saved to {output_path}")
        return True
    else:
        print(f"Error fetching NAIP: {response.status_code}")
        return False


# Use it
naip_file = data_dir / 'naip_imagery.tif'
print("Downloading NAIP imagery...")
success = get_naip_imagery(bbox, naip_file)

if success:
    # Check what we got
    with rasterio.open(naip_file) as src:
        print(f"Bands: {src.count}")
        print(f"Size: {src.width}x{src.height}")

## Part 3: Download Elevation Data (SRTM)

SRTM elevation data via Open-Elevation API (no authentication required).

In [ ]:
def get_srtm_elevation(bbox, output_path):
    """
    Fetch SRTM elevation data using Open-Elevation API
    Creates elevation grid from point queries
    """
    # Create grid of points within bbox
    lat_points = 100
    lon_points = 100

    lats = np.linspace(bbox[1], bbox[3], lat_points)
    lons = np.linspace(bbox[0], bbox[2], lon_points)

    lon_grid, lat_grid = np.meshgrid(lons, lats)
    elevation_grid = np.zeros((lat_points, lon_points))

    print(f"Fetching elevation for {lat_points}x{lon_points} grid...")

    # Query elevation API in batches
    base_url = "https://api.open-elevation.com/api/v1/lookup"

    locations = []
    for i in range(lat_points):
        for j in range(lon_points):
            locations.append({
                'latitude': lat_grid[i, j],
                'longitude': lon_grid[i, j]
            })

    # Split into batches to avoid timeout
    batch_size = 5000
    all_results = []

    for i in range(0, len(locations), batch_size):
        batch = locations[i:i + batch_size]
        response = requests.post(base_url, json={'locations': batch}, timeout=60)

        if response.status_code == 200:
            results = response.json()['results']
            all_results.extend(results)
            print(f"  Processed {len(all_results)}/{len(locations)} points")
        else:
            print(f"Error at batch {i}: {response.status_code}")
            return False

    # Fill elevation grid - flip vertically to match image orientation
    for idx, result in enumerate(all_results):
        i = idx // lon_points
        j = idx % lon_points
        elevation_grid[i, j] = result['elevation']

    # Flip array vertically so it matches NAIP orientation
    elevation_grid = np.flipud(elevation_grid)

    # Save as GeoTIFF
    transform = from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], lon_points, lat_points)

    with rasterio.open(
            output_path, 'w',
            driver='GTiff',
            height=lat_points,
            width=lon_points,
            count=1,
            dtype=elevation_grid.dtype,
            crs='EPSG:4326',
            transform=transform
    ) as dst:
        dst.write(elevation_grid, 1)

    return True


dem_file = data_dir / 'elevation.tif'
print("Downloading SRTM elevation data...")
success = get_srtm_elevation(bbox, dem_file)

if success and dem_file.exists():
    print(f"Elevation data saved to {dem_file}")
else:
    print("Elevation download failed")

## Part 4: Visualize NAIP Imagery

In [ ]:
if naip_file.exists():
    with rasterio.open(naip_file) as src:
        naip_data = src.read()

        print(f"NAIP Data Properties:")
        print(f"  Shape: {naip_data.shape}")
        print(f"  Bands: {src.count}")
        print(f"  Resolution: {src.res[0]:.6f}° x {src.res[1]:.6f}°")
        print(f"  CRS: {src.crs}")

        # Display RGB imagery
        plt.figure(figsize=(12, 10))

        # Use first 3 bands for RGB (handle RGBA if present)
        if src.count >= 3:
            rgb = np.moveaxis(naip_data[:3], 0, -1)
        else:
            rgb = np.moveaxis(naip_data, 0, -1)

        plt.imshow(rgb)
        plt.title('NAIP Aerial Imagery (1m resolution)', fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

## Part 5: Visualize Elevation Data

In [ ]:
if dem_file.exists():
    with rasterio.open(dem_file) as src:
        elevation_data = src.read(1)
        elevation_transform = src.transform
        elevation_crs = src.crs

        print(f"Elevation Data Properties:")
        print(f"  Shape: {elevation_data.shape}")
        print(f"  CRS: {src.crs}")
        print(f"  Elevation range: {elevation_data.min():.0f}m to {elevation_data.max():.0f}m")

        plt.figure(figsize=(12, 10))
        im = plt.imshow(elevation_data, cmap='terrain')
        plt.colorbar(im, label='Elevation (m)')
        plt.title('SRTM Elevation Data', fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

## Part 6: Align NAIP and Elevation Data

Resample elevation to match NAIP resolution for integrated analysis.

In [ ]:
if naip_file.exists() and dem_file.exists():
    with rasterio.open(naip_file) as naip_src:
        with rasterio.open(dem_file) as dem_src:
            # Prepare output array matching NAIP dimensions
            elevation_resampled = np.empty(
                (naip_src.height, naip_src.width),
                dtype=rasterio.float32
            )

            # Reproject elevation to match NAIP
            reproject(
                source=rasterio.band(dem_src, 1),
                destination=elevation_resampled,
                src_transform=dem_src.transform,
                src_crs=dem_src.crs,
                dst_transform=naip_src.transform,
                dst_crs=naip_src.crs,
                resampling=Resampling.bilinear
            )

            print(f"Elevation resampled to NAIP resolution")
            print(f"  NAIP shape: {naip_data.shape[1:]}")
            print(f"  Resampled elevation shape: {elevation_resampled.shape}")

## Part 7: Integrated Visualization

Display both datasets side-by-side and as overlay.

In [ ]:
if naip_file.exists() and dem_file.exists():
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))

    # NAIP imagery
    if naip_data.shape[0] >= 3:
        rgb = np.moveaxis(naip_data[:3], 0, -1)
    else:
        rgb = np.moveaxis(naip_data, 0, -1)

    axes[0].imshow(rgb)
    axes[0].set_title('NAIP Imagery (1m)', fontsize=12, fontweight='bold')
    axes[0].axis('off')

    # Elevation
    im1 = axes[1].imshow(elevation_resampled, cmap='terrain')
    axes[1].set_title('Elevation Resampled to 1m', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1], label='Elevation (m)')

    # Overlay: imagery with elevation contours
    axes[2].imshow(rgb)
    contours = axes[2].contour(elevation_resampled, levels=10, colors='white',
                               linewidths=1.5, alpha=0.7)
    axes[2].clabel(contours, inline=True, fontsize=8)
    axes[2].set_title('Combined: NAIP + Elevation Contours', fontsize=12, fontweight='bold')
    axes[2].axis('off')

    plt.suptitle('Multi-Source Geospatial Data Integration', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(data_dir / 'integrated_view.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Integrated visualization saved to {data_dir / 'integrated_view.png'}")

## Part 8: Fire Risk Assessment

Combine imagery and elevation to identify high-risk areas.

In [ ]:
if naip_file.exists() and dem_file.exists():

    # Read the data
    with rasterio.open(naip_file) as src:
        naip_data = src.read()

    with rasterio.open(dem_file) as src:
        elevation_data = src.read(1)

    # Resample elevation to match NAIP size
    from rasterio.warp import reproject, Resampling

    with rasterio.open(naip_file) as naip_src:
        with rasterio.open(dem_file) as dem_src:
            elevation_resampled = np.empty(
                (naip_src.height, naip_src.width),
                dtype=rasterio.float32
            )

            reproject(
                source=rasterio.band(dem_src, 1),
                destination=elevation_resampled,
                src_transform=dem_src.transform,
                src_crs=dem_src.crs,
                dst_transform=naip_src.transform,
                dst_crs=naip_src.crs,
                resampling=Resampling.bilinear
            )

    # ============================================
    # SIMPLE FIRE RISK MODEL
    # ============================================

    # 1. VEGETATION RISK (from RGB image)
    # Brown/dry areas = high risk
    # Green areas = medium risk (fuel but moist)
    # Water/rock = low risk

    red = naip_data[0].astype(float)
    green = naip_data[1].astype(float)
    blue = naip_data[2].astype(float)

    # Brown could be dry vegetation
    brownness = (red - green) / 255.0
    brownness = np.clip(brownness, 0, 1)  # Keep 0-1

    # Green could be vegetation
    greenness = (green - red - blue) / 255.0
    greenness = np.clip(greenness, 0, 1)

    # Vegetation risk: brown = very high, green = medium, other = low
    veg_risk = np.zeros_like(brownness)
    veg_risk = brownness * 0.9  # Dry vegetation = 90% risk
    veg_risk += greenness * 0.6  # Green vegetation = 50% risk
    veg_risk = np.clip(veg_risk, 0, 1)

    # 3. SLOPE RISK
    # Steep = fire spreads faster

    # Calculate slope (no complex math)
    dy, dx = np.gradient(elevation_resampled)
    slope = np.sqrt(dx ** 2 + dy ** 2)

    # Normalize slope: gentle = 0, steep = 1
    slope_risk = np.clip(slope / np.percentile(slope, 95), 0, 1)

    # 4. COMBINED RISK
    fire_risk = np.clip((veg_risk + slope_risk), 0, 1)

    # ============================================
    # VISUALIZE
    # ============================================

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # 1. RGB imagery
    if naip_data.shape[0] >= 3:
        rgb = np.moveaxis(naip_data[:3], 0, -1)
    else:
        rgb = naip_data[0]

    axes[0, 0].imshow(rgb)
    axes[0, 0].set_title('Satellite Imagery', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

    # 2. Elevation
    im1 = axes[0, 1].imshow(elevation_resampled, cmap='terrain')
    axes[0, 1].set_title('Elevation', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    plt.colorbar(im1, ax=axes[0, 1], label='Meters')

    # 3. Vegetation risk
    im2 = axes[0, 2].imshow(veg_risk, cmap='YlOrRd', vmin=0, vmax=1)
    axes[0, 2].set_title('Vegetation Risk\n(brown/dry = high)', fontsize=12, fontweight='bold')
    axes[0, 2].axis('off')
    plt.colorbar(im2, ax=axes[0, 2], label='Risk')

    # 5. Slope risk
    im4 = axes[1, 1].imshow(slope_risk, cmap='YlOrRd', vmin=0, vmax=1)
    axes[1, 1].set_title('Slope Risk\n(steep = high)', fontsize=12, fontweight='bold')
    axes[1, 1].axis('off')
    plt.colorbar(im4, ax=axes[1, 1], label='Risk')

    # 6. FINAL FIRE RISK
    im5 = axes[1, 2].imshow(fire_risk, cmap='hot_r', vmin=0, vmax=1)
    axes[1, 2].set_title('Fire Risk', fontsize=14, fontweight='bold')
    axes[1, 2].axis('off')
    plt.colorbar(im5, ax=axes[1, 2], label='Risk Score')

    plt.suptitle('Fire Risk Analysis', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(data_dir / 'fire_risk_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()